¿Cómo lidiar con datos faltantes en tus DataFrames?
Es muy común que nuestros DataFrames presenten datos faltantes, antes de empezar a procesar nuestros DataFrames veamos un poco en qué consisten los objetos NaN (Not a Number).

In [1]:
import numpy as np
import pandas as pd
np.nan

nan

Propiedades matematicas de un NaN

In [2]:
np.nan + 0

nan

In [3]:
np.nan > 0

False

La versión 1.0 de pandas incluye un nuevo objeto NA, que es mucho más
general pues, ademas de interactuar con números, tambien puede hacerlo con
cadenas de texto u otras varaibles como las de tipo booleano. Si quieres que
esta nueva definición este incluida entre tus cálculos usa:

In [6]:
pd.options.mode.use_inf_as_na = True
pd.NA +'Hola mundo'

<NA>

In [7]:
pd.NA | False

<NA>

In [8]:
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3), columns=['a', 'b', 'c'])
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14


In [9]:
df['d'] = np.nan
df['e'] = np.arange(15, 20)
df.loc[5,:] = pd.NA
df.loc[4,'a'] = pd.NA
df.loc[0,'d'] = 1
df.loc[5,'d'] = 10
df

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,NaN,16.0
2,6.0,7.0,8.0,NaN,17.0
3,9.0,10.0,11.0,NaN,18.0
4,<NA>,13.0,14.0,NaN,19.0
5,<NA>,<NA>,<NA>,10,<NA>


In [10]:
df.isnull()

,a,b,c,d,e
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,False,True,False
3,False,False,False,True,False
4,True,False,False,True,False
5,True,True,True,False,True


Nulos por columna:

In [11]:
df.isnull().sum()

a    2
b    1
c    1
d    4
e    1
dtype: int64

Nulos por fila:

In [12]:
df.notnull().sum(axis=1)

0    5
1    4
2    4
3    4
4    3
5    1
dtype: int64

Todos los elementos nulos de nuestro DataFrame:

In [13]:
df.size-df.isnull().sum().sum()

21

Mostrar filas no nulas de las columna 'a'

In [14]:
df[df['a'].notnull()]

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,NaN,16.0
2,6.0,7.0,8.0,NaN,17.0
3,9.0,10.0,11.0,NaN,18.0


Elimnar las filas con algun registro faltante

In [15]:
df.dropna()

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0


In [16]:
df[['a']].dropna()

,a
0,0.0
1,3.0
2,6.0
3,9.0


Con fillna podremos reemplazarlas por el valor que querramos, en este caso 0.

In [17]:
df.fillna(0)

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,0,16.0
2,6.0,7.0,8.0,0,17.0
3,9.0,10.0,11.0,0,18.0
4,0.0,13.0,14.0,0,19.0
5,0.0,0.0,0.0,10,0.0


Si quisieramos remplazar con el valor siguiente (ultimo valor asignado), usamos method="ffill":

In [18]:
df.fillna(method="ffill")

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,1,16.0
2,6.0,7.0,8.0,1,17.0
3,9.0,10.0,11.0,1,18.0
4,9.0,13.0,14.0,1,19.0
5,9.0,13.0,14.0,10,19.0


Si quisieramos remplazar con el valor previo (valor posterior que encuentre primero) usamos method="bfill":

In [19]:
df.fillna(method="bfill")

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,10,16.0
2,6.0,7.0,8.0,10,17.0
3,9.0,10.0,11.0,10,18.0
4,<NA>,13.0,14.0,10,19.0
5,<NA>,<NA>,<NA>,10,<NA>


se puede aplicar con las filas usando axis=1:

In [20]:
df.fillna(method="bfill",axis=1)

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,16.0,16.0
2,6.0,7.0,8.0,17.0,17.0
3,9.0,10.0,11.0,18.0,18.0
4,13.0,13.0,14.0,19.0,19.0
5,10.0,10.0,10.0,10.0,<NA>


Remplazar con otra serie o DF respetando los numero de columnas

In [21]:
fill = pd.Series([100, 101, 102])
fill

0    100
1    101
2    102
dtype: int64

In [23]:
df['d'] = df['d'].fillna(fill)
df

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,101.0,16.0
2,6.0,7.0,8.0,102.0,17.0
3,9.0,10.0,11.0,NaN,18.0
4,<NA>,13.0,14.0,NaN,19.0
5,<NA>,<NA>,<NA>,10.0,<NA>


In [24]:
df.fillna(df.median())

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,101.0,16.0
2,6.0,7.0,8.0,102.0,17.0
3,9.0,10.0,11.0,55.5,18.0
4,4.5,13.0,14.0,55.5,19.0
5,4.5,7.0,8.0,10.0,17.0


Pandas también puede interporlar los valores faltanes calculando el valor que puede haber existido en el medio.

In [25]:
df_d = pd.concat([df[['d']], df[['d']].interpolate()],axis=1)
df_d.columns = ['d_antes','d_interpolado']
df_d

,d_antes,d_interpolado
0,1.0,1.000000
1,101.0,101.000000
2,102.0,102.000000
3,NaN,71.333333
4,NaN,40.666667
5,10.0,10.000000
